In [1]:
#image compression 

In [5]:
import numpy as np
import math as m
import cv2 as cv
import matplotlib.pyplot as plt

# get and convert img to YCbCr
def get_img(path):
    img = cv.imread(path)
    img_ = cv.cvtColor(img, cv.COLOR_RGB2YCrCb)
    return img_

img = get_img('duck.jpg')

# block divider
def block_divider(img):
    height_index = img.shape[0]//8-1
    weight_index = img.shape[1]//8-1
    blocks = []
    for n in range(3):
        for i in range(weight_index):
            for j in range(height_index):
                blocks.append(img[i*8:(i+1)*8, j*8:(j+1)*8, n])
    return blocks
# DCT Forward
def DCT_Forward(block):
    N = block.shape[0]

    output = np.zeros(block.shape, dtype = 'double')
    for i in range(N):
        for j in range(N):
            #caculate FDCT
            Sum = 0

            for k in range(N):
                for l in range(N):
                    Sum = Sum + ( block[k,l]*m.cos( ((2*k)+1)*(i*m.pi)/16)*m.cos( ((2*l)+1)*(j*m.pi)/16))
            Cu = m.sqrt(1/2) if i == 0 else 1
            Cv = m.sqrt(1/2) if j == 0 else 1
            output[i,j]= (1/4)*Cu*Cv*Sum #output[i,j] be4 Quantazation
    return output

#quantazation matrix
QM = np.asarray([[16 ,11 ,10 ,16 ,24 ,40 ,51 ,61],
       [12 ,12 ,14 ,19 ,26 ,58 ,60 ,55],
       [14 ,13 ,16 ,24 ,40 ,57 ,69 ,56],
       [14 ,17 ,22 ,29 ,51 ,87 ,80 ,62],
       [18 ,22 ,37 ,56 ,68 ,109 ,103 ,77],
       [24 ,35 ,55 ,64 ,81 ,104 ,113 ,92],
       [49 ,64 ,78 ,87 ,103 ,121 ,120 ,101],
       [72 ,92 ,95 ,98 ,112 ,100 ,103 ,99]])
# Quantization
def quant(block, QM = QM):
    N = block.shape[0]
    output = np.zeros(block.shape, dtype = 'int16')
    for i in range(N):
        for j in range(N):
            output[i,j] = block[i,j]//QM[i,j]
    return output
# HuffMan encoding

#read blocks
def read_block_zig_zac(block):
    sequence =[]
    sequence.append(block[0,0])
    sequence.append(block[0,1])
    sequence.append(block[1,0])
    sequence.append(block[2,0])
    sequence.append(block[1,1])
    sequence.append(block[0,2])
    sequence.append(block[0,3])
    sequence.append(block[1,2])
    sequence.append(block[2,1])
    sequence.append(block[3,0])
    sequence.append(block[4,0])
    sequence.append(block[3,1])
    sequence.append(block[2,2])
    sequence.append(block[1,3])
    sequence.append(block[0,4])
    sequence.append(block[0,5])
    sequence.append(block[1,4])
    sequence.append(block[2,3])
    sequence.append(block[3,2])
    sequence.append(block[4,1])
    sequence.append(block[5,0])
    sequence.append(block[6,0])
    sequence.append(block[5,1])
    sequence.append(block[4,2])
    sequence.append(block[3,3])
    sequence.append(block[2,4])
    sequence.append(block[1,5])
    sequence.append(block[0,6])
    sequence.append(block[0,7])
    sequence.append(block[1,6])
    sequence.append(block[2,5])
    sequence.append(block[3,4])
    sequence.append(block[4,3])
    sequence.append(block[5,2])
    sequence.append(block[6,1])
    sequence.append(block[7,0])
    sequence.append(block[7,1])
    sequence.append(block[6,2])
    sequence.append(block[5,3])
    sequence.append(block[4,4])
    sequence.append(block[3,5])
    sequence.append(block[2,6])
    sequence.append(block[1,7])
    sequence.append(block[2,7])
    sequence.append(block[3,6])
    sequence.append(block[4,5])
    sequence.append(block[5,4])
    sequence.append(block[6,3])
    sequence.append(block[7,2])
    sequence.append(block[7,3])
    sequence.append(block[6,4])
    sequence.append(block[5,5])
    sequence.append(block[4,6])
    sequence.append(block[3,7])
    sequence.append(block[4,7])
    sequence.append(block[5,6])
    sequence.append(block[6,5])
    sequence.append(block[7,4])
    sequence.append(block[7,5])
    sequence.append(block[6,6])
    sequence.append(block[5,7])
    sequence.append(block[6,7])
    sequence.append(block[7,6])
    sequence.append(block[7,7])
    return sequence

def DC_coefficient(DC_pre, DC):
    dif = DC_pre - DC
    cat = 0
    if dif ==0:
        cat = 0
    else:
        cat = m.ceil(m.log2(abs(dif)))
    pos = 1 if dif>0 else 0
    return cat,pos,dif

def AC_coefficient(block):
    sequence = []

    count = 0
    for i in range(1,len(block)):
        if block[i] == 0:
            count += 1
            if count == 16:
                sequence.append('ZRL')
                count = 0
        else:
            cat = m.ceil(m.log2(abs(block[i])))
            sequence.append([count, cat, block[i] ])
            count = 0
    return sequence

def HuffMan_Coding(pre_DC, block):
    #this block is list that 've been readed as zigzag'
    sequence = []
    DC_coe = DC_coefficient(pre_DC, block[0])
    sequence.append([DC_coe[0], DC_coe[1], DC_coe[2]]) #cat-pos-DC
    AC_coe = AC_coefficient(block)
    for i in range(len(AC_coe)):
        sequence.append(AC_coe[i])
    return sequence,DC_coe[2] #new DC

In [7]:
blocks = block_divider(img)
test_block = blocks[0]
print("number of blocks:")
print(len(blocks))
print("base block:")
print(test_block)

test_block = DCT_Forward(test_block)
print("DCT block")
print(test_block)

test_block = quant(test_block)
print("quantazation block")
print(test_block)

list_block = read_block_zig_zac(test_block)
print("list_block")
print(list_block)

coefficient = HuffMan_Coding(0,list_block)
print("the coded:")
print(coefficient[0])
print("new DC:")
print(coefficient[1])


number of blocks:
29040
base block:
[[ 50  54  61  67  67  59  51  49]
 [ 49  54  62  71  72  65  61  60]
 [ 42  49  61  72  74  69  67  68]
 [ 34  44  60  72  74  70  69  71]
 [ 28  40  59  72  74  69  69  73]
 [ 21  34  52  66  68  67  71  79]
 [ 20  32  49  61  65  71  84  98]
 [ 27  37  53  65  72  81 102 120]]
DCT block
[[ 4.90875000e+02 -1.03890941e+02 -4.25088988e+01 -2.48095415e+01
   9.37500000e+00 -3.83629980e-01 -3.87007937e-01  2.72655762e-01]
 [-1.04406393e+01  7.22819287e+01 -1.77391910e+01  1.57916899e+01
  -2.95145886e-01 -7.37438605e-01 -3.45471476e-01  3.56869007e-02]
 [ 6.98994689e+00 -5.61909476e+00  1.82289998e+01  1.01924461e-01
   2.80213456e-02 -3.24743731e-01 -1.91941738e-01  1.57311973e-01]
 [-2.36451223e+01  1.50176745e+01 -9.07859262e+00  5.18462946e-01
  -3.13504456e-01 -7.43902378e-01  6.17748885e-01 -6.05922650e-01]
 [ 7.37500000e+00 -3.39899986e-01  2.80213456e-02 -2.28345827e-01
  -1.25000000e-01  2.35540357e-02  3.94290254e-01  8.02665179e-01]
 [-9.163

In [53]:

def Bin_encoding(sequence):
    bit_seq = ''
    DC_coe = sequence[0]
    bit_seq = bit_seq + str(DC_ditc[DC_coe[0]])
    if(DC_coe[2]< 0):
        binn = str(bin(DC_coe[2]))[3:]
#         print('binn')
#         print(binn)
        new_bin = ""
        for i in range(len(binn)):
            if binn[i] == '0':
                new_bin = new_bin + '1'
            else:
                new_bin = new_bin + '0'
#         print("new bin")
#         print(new_bin)
        bit_seq = bit_seq + new_bin
    for i in enumerate(sequence[1:]):
        cat = 
    return bit_seq

In [55]:
DC_ditc = {
    0:'00',
    1:'010',
    2:'011',
    3:'100',
    4:'101',
    5:'110',
    6:'1110',
    7:'11110',
    8:'111110',
    9:'1111110',
    10:'11111110',
    11:'111111110'
}

#testing 2
seq = Bin_encoding(coefficient[0])
print(seq)
for i in enumerate(coefficient):
    print(i)

binn
11110
new bin
00001
11000001
(0, [[5, 0, -30], [0, 4, -10], [0, 0, -1], [1, 3, 6], [0, 3, -5], [0, 1, -2], [0, 1, -2], [0, 0, -1], [0, 1, -2], [2, 0, 1], [2, 0, -1], [0, 0, -1], [1, 0, -1], [0, 0, -1], [0, 0, -1], [1, 0, -1], [3, 0, -1], [0, 0, -1], [1, 0, -1], [0, 0, -1], [0, 0, -1], [0, 0, -1], [2, 0, -1], [1, 0, -1], [0, 0, -1], [0, 0, -1], [0, 0, -1], [0, 0, -1], [4, 0, -1], [1, 0, -1], [0, 0, -1], [1, 0, -1], [1, 0, -1], [2, 0, -1], [1, 0, -1], [1, 0, -1], [0, 0, -1]])
(1, -30)
